In [3]:
import torch
import numpy as np
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel

from generate import *

device = 'cuda'

model = AutoModel.from_pretrained('GSAI-ML/LLaDA-8B-Instruct', trust_remote_code=True, torch_dtype=torch.bfloat16).to(device).eval()
tokenizer = AutoTokenizer.from_pretrained('GSAI-ML/LLaDA-8B-Instruct', trust_remote_code=True)

In [5]:
prompt = 'In the sentence: "The doctor phoned the nurse because she was late for the morning shift", who was late for the morning shift?'

# Add special tokens for the Instruct model. The Base model does not require the following two lines.
m = [{"role": "user", "content": prompt}, ]
prompt = tokenizer.apply_chat_template(m, add_generation_prompt=True, tokenize=False)

input_ids = tokenizer(prompt)['input_ids']
input_ids = torch.tensor(input_ids).to(device).unsqueeze(0)

out = generate(model, input_ids, steps=64, gen_length=64, block_length=32, temperature=0., cfg_scale=0., remasking='low_confidence')
print(tokenizer.batch_decode(out[:, input_ids.shape[1]:], skip_special_tokens=True)[0])

In the sentence "The doctor phoned the nurse because she was late for the morning shift," the nurse was late for the morning shift.


In [4]:
input_ids = tokenizer("Once upon a time, a PhD student", return_tensors='pt').input_ids.to(device)  # [1, seq_len]

for i in range(1):
    out = generate(model, input_ids, steps=64, gen_length=64, block_length=32, temperature=0.2, cfg_scale=0., remasking='low_confidence')
    generated_text = tokenizer.batch_decode(out, skip_special_tokens=True)[0]
    print(generated_text)   

Once upon a time, a PhD student named Alex was struggling to find a unique topic for their dissertation. One day, they stumbled upon a fascinating article about the impact of social media on mental health.
Once upon a time, a PhD student named Alex was struggling to find a unique topic for their dissertation. One day, they stumbled upon an old, dusty book in the library that piqued their interest. As they delved deeper, Alex discovered a hidden world of knowledge that would change their life forever.

The rest is the story.
Once upon a time, a PhD student named Alex was working on a project to study the effects of climate change on marine ecosystems. They found that the warming of the ocean was causing coral bleaching and the decline of marine species.


KeyboardInterrupt: 

In [ ]:
input_ids = tokenizer("Once upon a time, ", return_tensors='pt').input_ids.to(device)  # [1, seq_len]

out = generate(model, input_ids, steps=64, gen_length=64, block_length=32, temperature=0., cfg_scale=0., remasking='low_confidence')
generated_text = tokenizer.batch_decode(out, skip_special_tokens=True)[0]
print(generated_text)

Once upon a time, 
there was a little girl named Lily. 
She loved to play in the park, 
and run with her friends. 
One day,


In [3]:
prompt = "Lily can run 12 kilometers per hour for 4 hours. After that, she runs 6 kilometers per hour. How many kilometers can she run in 8 hours?"

# Add special tokens for the Instruct model. The Base model does not require the following two lines.
m = [{"role": "user", "content": prompt}, ]
prompt = tokenizer.apply_chat_template(m, add_generation_prompt=True, tokenize=False)

input_ids = tokenizer(prompt)['input_ids']
input_ids = torch.tensor(input_ids).to(device).unsqueeze(0)

out = generate(model, input_ids, steps=64, gen_length=64, block_length=32, temperature=0., cfg_scale=0., remasking='low_confidence')
print(tokenizer.batch_decode(out[:, input_ids.shape[1]:], skip_special_tokens=True)[0])

In the first 4 hours, Lily runs $12 \times 4 = 48$ kilometers. In the next 4 hours, she runs $6 \times 4 = 24$ kilometers. The total is $48+24=\boxed{72}$ kilometers.
